In [1]:
'''
This file plots CMIP5 RCP - HIST over South Australia
/Users/earl/Desktop/Yang
and places the outputs in
/Users/earl/Dropbox/CMIP5/figures

Earl Duran 
created: 19-Mar-18
e.duran@unsw.edu.au
'''

import cosima_cookbook as cc
import os
import xarray as xr
import numpy as np
from mpl_toolkits.basemap import Basemap
import matplotlib.pyplot as plt
import matplotlib
import sys
import pickle
import itertools
def find_nearest_index(array, value):
    return int((np.abs(array - value)).argmin())
from scipy import interpolate
from scipy import stats
from dask.distributed import Client

import cartopy.crs as ccrs
import cartopy.feature as cft
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
def arange(start,end,stride=1):
    return np.arange(start,end+0.00000001,stride)

import sys, os
sys.path.append(os.path.join(os.getcwd(), '..'))  # so we can import ../exptdata
import exptdata
print('Available exptdata keys: ', [k for k in exptdata.exptdict.keys()])

session = cc.database.create_session()

Available exptdata keys:  ['1deg', '025deg', '01deg']


In [2]:
%%javascript
IPython.notebook.kernel.execute('nb_name = ' + '"' + IPython.notebook.notebook_name + '"')

<IPython.core.display.Javascript object>

In [3]:
client = Client()
client

/g/data3/hh5/public/apps/miniconda3/envs/analysis3-19.07/lib/python3.6/site-packages/distributed/dashboard/core.py:72: UserWarning: 
Port 8787 is already in use. 
Perhaps you already have a cluster running?
Hosting the diagnostics dashboard on a random port instead.
  warnings.warn("\n" + msg)


Client Scheduler: tcp://127.0.0.1:41467 Dashboard: http://127.0.0.1:46019/status,Cluster Workers: 4 Cores: 8 Memory: 33.67 GB


In [4]:
# input_path = '/g/data/e14/erd561/NOAA_OI_SST_V2/'

output_path = '/g/data/e14/erd561/Australia_3/'

In [5]:
%%time
latS = -80
latN = 20
lonW = 90-360
lonE = 180-360
ekey='1deg'
expt = exptdata.exptdict[ekey]['expt']
print(expt)

tx_trans_z0_1_mod_month = cc.querying.getvar(expt=expt,
                                     session=session,
                                     ncfile='ocean_month.nc',
                                     variable='tx_trans_int_z').\
sel(yt_ocean=slice(latS,latN)).\
sel(xu_ocean=slice(lonW,lonE))
print(tx_trans_z0_1_mod_month)

tx_trans_z0_1_mod_month.load()
print(tx_trans_z0_1_mod_month)

xu_ocean = tx_trans_z0_1_mod_month.xu_ocean
xu_ocean_corrected = xu_ocean + 360
tx_trans_z0_1_mod_month = tx_trans_z0_1_mod_month.assign_coords(xu_ocean=xu_ocean_corrected)
print(tx_trans_z0_1_mod_month)

tx_trans_z0_1_mod = tx_trans_z0_1_mod_month.groupby('time.year').mean('time')
print(tx_trans_z0_1_mod)

tx_trans_z0_1_mod = tx_trans_z0_1_mod.sel(year=slice(1993+60*4,2017+60*4))
print(tx_trans_z0_1_mod)

tx_trans_z0_1_mod = tx_trans_z0_1_mod.cumsum('yt_ocean')
print(tx_trans_z0_1_mod)

1deg_jra55v13_iaf_spinup1_B1


/g/data3/hh5/public/apps/miniconda3/envs/analysis3-19.07/lib/python3.6/site-packages/xarray/backends/api.py:783: FutureWarning: In xarray version 0.13 `auto_combine` will be deprecated.
  coords=coords)
/g/data3/hh5/public/apps/miniconda3/envs/analysis3-19.07/lib/python3.6/site-packages/xarray/backends/api.py:783: FutureWarning: The datasets supplied have global dimension coordinates. You may want
to use the new `combine_by_coords` function (or the
`combine='by_coords'` option to `open_mfdataset` to order the datasets
before concatenation. Alternatively, to continue concatenating based
on the order the datasets are supplied in in future, please use the
new `combine_nested` function (or the `combine='nested'` option to
open_mfdataset).
  coords=coords)


<xarray.DataArray 'tx_trans_int_z' (time: 3600, yt_ocean: 186, xu_ocean: 91)>
dask.array<shape=(3600, 186, 91), dtype=float32, chunksize=(1, 186, 91)>
Coordinates:
  * yt_ocean  (yt_ocean) float64 -77.88 -77.63 -77.38 ... 17.85 18.59 19.36
  * xu_ocean  (xu_ocean) float64 -270.0 -269.0 -268.0 ... -182.0 -181.0 -180.0
  * time      (time) object 1958-01-16 12:00:00 ... 2257-12-16 12:00:00
Attributes:
    long_name:      T-cell i-mass transport vertically summed
    units:          kg/s
    valid_range:    [-1.e+20  1.e+20]
    cell_methods:   time: mean
    time_avg_info:  average_T1,average_T2,average_DT
    coordinates:    geolon_c geolat_t
<xarray.DataArray 'tx_trans_int_z' (time: 3600, yt_ocean: 186, xu_ocean: 91)>
array([[[          nan,           nan, ...,           nan,           nan],
        [          nan,           nan, ...,  2.283202e+07,  7.379601e+07],
        ...,
        [ 1.990314e+09,  1.410030e+09, ...,  1.026600e+09,  6.957160e+08],
        [ 1.425035e+09,  9.733433e

/g/data3/hh5/public/apps/miniconda3/envs/analysis3-19.07/lib/python3.6/site-packages/xarray/core/nanops.py:160: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis=axis, dtype=dtype)


<xarray.DataArray 'tx_trans_int_z' (year: 300, yt_ocean: 186, xu_ocean: 91)>
array([[[          nan,           nan, ...,           nan,           nan],
        [          nan,           nan, ...,  4.638585e+07,  6.966981e+07],
        ...,
        [ 1.385899e+08,  1.310004e+08, ..., -8.816944e+08, -9.812039e+08],
        [ 1.156018e+08,  8.867859e+07, ..., -5.440641e+08, -4.561756e+08]],

       [[          nan,           nan, ...,           nan,           nan],
        [          nan,           nan, ...,  5.290094e+07,  7.835662e+07],
        ...,
        [ 1.771059e+08,  1.079837e+08, ..., -2.300978e+09, -2.256453e+09],
        [ 1.277534e+08,  6.953747e+07, ..., -2.056907e+09, -2.014724e+09]],

       ...,

       [[          nan,           nan, ...,           nan,           nan],
        [          nan,           nan, ...,  7.530934e+07,  1.004466e+08],
        ...,
        [ 2.451205e+08,  1.646024e+08, ..., -1.162387e+09, -1.069488e+09],
        [ 1.572961e+08,  1.103236e+08, ...

In [6]:
year = tx_trans_z0_1_mod.year
lat_1_mod = tx_trans_z0_1_mod.yt_ocean
lon_1_mod = tx_trans_z0_1_mod.xu_ocean

tx_trans_z0_1_trans_mod = tx_trans_z0_1_mod
print(np.shape(tx_trans_z0_1_trans_mod))
tx_trans_z0_1_slope_mod = xr.DataArray(
    np.zeros([np.shape(lat_1_mod)[0], np.shape(lon_1_mod)[0]]), dims=(
    'lat_1', 'lon_1'), coords=[lat_1_mod, lon_1_mod], name='tx_trans_z0_1')
tx_trans_z0_1_p_value_mod = xr.DataArray(
    np.zeros([np.shape(lat_1_mod)[0], np.shape(lon_1_mod)[0]]), dims=(
    'lat_1', 'lon_1'), coords=[lat_1_mod, lon_1_mod], name='tx_trans_z0_1')
tx_trans_z0_1_std_err_mod = xr.DataArray(
    np.zeros([np.shape(lat_1_mod)[0], np.shape(lon_1_mod)[0]]), dims=(
    'lat_1', 'lon_1'), coords=[lat_1_mod, lon_1_mod], name='tx_trans_z0_1')
for iid, i in enumerate(lat_1_mod):
    for jid, j in enumerate(lon_1_mod):
        tx_trans_z0_1_slope_mod[iid, jid], _, _, tx_trans_z0_1_p_value_mod[iid, jid], tx_trans_z0_1_std_err_mod[iid, jid] = \
        stats.linregress(year, tx_trans_z0_1_trans_mod[:, iid, jid])
    print('lat_1_mod ' + str(np.array(i)))
print(tx_trans_z0_1_slope_mod)

(25, 186, 91)
lat_1_mod -77.87662337662337
lat_1_mod -77.62971340045422
lat_1_mod -77.38170670967813
lat_1_mod -77.13229193169715
lat_1_mod -76.88053325031572
lat_1_mod -76.62612553775392
lat_1_mod -76.36814128806307
lat_1_mod -76.1062857353073
lat_1_mod -75.8396437697329
lat_1_mod -75.5679350353177
lat_1_mod -75.29026082195642
lat_1_mod -75.0063591357146
lat_1_mod -74.71535156045195
lat_1_mod -74.41699829429905
lat_1_mod -74.11044497434412
lat_1_mod -73.7954776730756
lat_1_mod -73.47126968000356
lat_1_mod -73.13763645207749
lat_1_mod -72.79378234640026
lat_1_mod -72.43955551893697
lat_1_mod -72.07419460280451
lat_1_mod -71.6975835499315
lat_1_mod -71.30899824976896
lat_1_mod -70.9083613089407
lat_1_mod -70.49498860561721
lat_1_mod -70.06884400260856
lat_1_mod -69.6292858330716
lat_1_mod -69.17632154294074
lat_1_mod -68.70935410409521
lat_1_mod -68.22843658249317
lat_1_mod -67.73301847540898
lat_1_mod -67.2232002021303
lat_1_mod -66.69847936238045
lat_1_mod -66.15900514695375
lat_1_mod

In [7]:
%%time
latS = -80
latN = 20
lonW = 90-360
lonE = 180-360
ekey='025deg'
expt = exptdata.exptdict[ekey]['expt']
print(expt)

tx_trans_z0_025_mod_month = cc.querying.getvar(expt=expt,
                                     session=session,
                                     ncfile='ocean_month.nc',
                                     variable='tx_trans_int_z').\
sel(yt_ocean=slice(latS,latN)).\
sel(xu_ocean=slice(lonW,lonE))
print(tx_trans_z0_025_mod_month)

tx_trans_z0_025_mod_month.load()
print(tx_trans_z0_025_mod_month)

xu_ocean = tx_trans_z0_025_mod_month.xu_ocean
xu_ocean_corrected = xu_ocean + 360
tx_trans_z0_025_mod_month = tx_trans_z0_025_mod_month.assign_coords(xu_ocean=xu_ocean_corrected)
print(tx_trans_z0_025_mod_month)

tx_trans_z0_025_mod = tx_trans_z0_025_mod_month.groupby('time.year').mean('time')
print(tx_trans_z0_025_mod)

tx_trans_z0_025_mod = tx_trans_z0_025_mod.sel(year=slice(1993+60*4,2017+60*4))
print(tx_trans_z0_025_mod)

tx_trans_z0_025_mod = tx_trans_z0_025_mod.cumsum('yt_ocean')
print(tx_trans_z0_025_mod)

025deg_jra55v13_iaf_gmredi6


/g/data3/hh5/public/apps/miniconda3/envs/analysis3-19.07/lib/python3.6/site-packages/xarray/backends/api.py:783: FutureWarning: In xarray version 0.13 `auto_combine` will be deprecated.
  coords=coords)
/g/data3/hh5/public/apps/miniconda3/envs/analysis3-19.07/lib/python3.6/site-packages/xarray/backends/api.py:783: FutureWarning: The datasets supplied have global dimension coordinates. You may want
to use the new `combine_by_coords` function (or the
`combine='by_coords'` option to `open_mfdataset` to order the datasets
before concatenation. Alternatively, to continue concatenating based
on the order the datasets are supplied in in future, please use the
new `combine_nested` function (or the `combine='nested'` option to
open_mfdataset).
  coords=coords)


<xarray.DataArray 'tx_trans_int_z' (time: 3600, yt_ocean: 569, xu_ocean: 361)>
dask.array<shape=(3600, 569, 361), dtype=float32, chunksize=(1, 529, 361)>
Coordinates:
  * yt_ocean  (yt_ocean) float64 -79.92 -79.81 -79.7 -79.6 ... 19.49 19.72 19.96
  * xu_ocean  (xu_ocean) float64 -270.0 -269.8 -269.5 ... -180.5 -180.2 -180.0
  * time      (time) object 1958-01-14 12:00:00 ... 2257-12-14 12:00:00
Attributes:
    long_name:      T-cell i-mass transport vertically summed
    units:          kg/s
    valid_range:    [-1.e+20  1.e+20]
    cell_methods:   time: mean
    time_avg_info:  average_T1,average_T2,average_DT
    coordinates:    geolon_c geolat_t
<xarray.DataArray 'tx_trans_int_z' (time: 3600, yt_ocean: 569, xu_ocean: 361)>
array([[[          nan,           nan, ...,           nan,           nan],
        [          nan,           nan, ...,           nan,           nan],
        ...,
        [ 2.122169e+09,  2.034380e+09, ...,  8.920833e+08,  7.217557e+08],
        [ 1.355444e+09,  

/g/data3/hh5/public/apps/miniconda3/envs/analysis3-19.07/lib/python3.6/site-packages/xarray/core/nanops.py:160: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis=axis, dtype=dtype)


<xarray.DataArray 'tx_trans_int_z' (year: 300, yt_ocean: 569, xu_ocean: 361)>
array([[[          nan,           nan, ...,           nan,           nan],
        [          nan,           nan, ...,           nan,           nan],
        ...,
        [-2.102862e+08, -1.704806e+08, ...,  2.889519e+08,  2.573828e+08],
        [-2.014383e+08, -1.724796e+08, ...,  1.811448e+08,  2.039236e+08]],

       [[          nan,           nan, ...,           nan,           nan],
        [          nan,           nan, ...,           nan,           nan],
        ...,
        [-2.239788e+08, -1.476704e+08, ..., -2.220990e+08, -3.680419e+08],
        [-1.746288e+08, -1.558761e+08, ..., -4.250644e+08, -4.547937e+08]],

       ...,

       [[          nan,           nan, ...,           nan,           nan],
        [          nan,           nan, ...,           nan,           nan],
        ...,
        [ 7.370502e+07,  1.149721e+08, ..., -1.117496e+08, -1.172492e+08],
        [ 1.808861e+07,  4.200145e+07, ..

In [8]:
lat_025_mod = tx_trans_z0_025_mod.yt_ocean
lon_025_mod = tx_trans_z0_025_mod.xu_ocean

tx_trans_z0_025_trans_mod = tx_trans_z0_025_mod
print(np.shape(tx_trans_z0_025_trans_mod))
tx_trans_z0_025_slope_mod = xr.DataArray(
    np.zeros([np.shape(lat_025_mod)[0], np.shape(lon_025_mod)[0]]), dims=(
    'lat_025', 'lon_025'), coords=[lat_025_mod, lon_025_mod], name='tx_trans_z0_025')
tx_trans_z0_025_p_value_mod = xr.DataArray(
    np.zeros([np.shape(lat_025_mod)[0], np.shape(lon_025_mod)[0]]), dims=(
    'lat_025', 'lon_025'), coords=[lat_025_mod, lon_025_mod], name='tx_trans_z0_025')
tx_trans_z0_025_std_err_mod = xr.DataArray(
    np.zeros([np.shape(lat_025_mod)[0], np.shape(lon_025_mod)[0]]), dims=(
    'lat_025', 'lon_025'), coords=[lat_025_mod, lon_025_mod], name='tx_trans_z0_025')
for iid, i in enumerate(lat_025_mod):
    for jid, j in enumerate(lon_025_mod):
        tx_trans_z0_025_slope_mod[iid, jid], _, _, tx_trans_z0_025_p_value_mod[iid, jid], tx_trans_z0_025_std_err_mod[iid, jid] = \
        stats.linregress(year, tx_trans_z0_025_trans_mod[:, iid, jid])
    print('lat_025_mod ' + str(np.array(i)))
print(tx_trans_z0_025_slope_mod)

(25, 569, 361)
lat_025_mod -79.91541635258267
lat_025_mod -79.80981776337775
lat_025_mod -79.70421917417283
lat_025_mod -79.59862058496792
lat_025_mod -79.493021995763
lat_025_mod -79.38742340655809
lat_025_mod -79.28182481735317
lat_025_mod -79.17622622814825
lat_025_mod -79.07062763894334
lat_025_mod -78.96502904973842
lat_025_mod -78.8594304605335
lat_025_mod -78.75383187132859
lat_025_mod -78.64823328212367
lat_025_mod -78.54263469291875
lat_025_mod -78.43703610371384
lat_025_mod -78.33143751450892
lat_025_mod -78.225838925304
lat_025_mod -78.12024033609909
lat_025_mod -78.01464174689417
lat_025_mod -77.90904315768925
lat_025_mod -77.80344456848434
lat_025_mod -77.69784597927942
lat_025_mod -77.5922473900745
lat_025_mod -77.48664880086959
lat_025_mod -77.38105021166467
lat_025_mod -77.27545162245976
lat_025_mod -77.16985303325484
lat_025_mod -77.06425444404992
lat_025_mod -76.958655854845
lat_025_mod -76.85305726564009
lat_025_mod -76.74745867643517
lat_025_mod -76.64186008723026
l

lat_025_mod -48.51019821855441
lat_025_mod -48.34430677060639
lat_025_mod -48.17787182793927
lat_025_mod -48.01089660539825
lat_025_mod -47.84337719706126
lat_025_mod -47.67531682924737
lat_025_mod -47.50671165403836
lat_025_mod -47.33756491026068
lat_025_mod -47.16787280849464
lat_025_mod -46.99763860110834
lat_025_mod -46.82685855767941
lat_025_mod -46.655535945152195
lat_025_mod -46.48366709260396
lat_025_mod -46.31125528258984
lat_025_mod -46.1382969041921
lat_025_mod -45.96479525661064
lat_025_mod -45.79074678944065
lat_025_mod -45.6161548195597
lat_025_mod -45.441015857585846
lat_025_mod -45.26533323910572
lat_025_mod -45.089103536271594
lat_025_mod -44.91233010440849
lat_025_mod -44.735009577715175
lat_025_mod -44.557145332281756
lat_025_mod -44.37873406486607
lat_025_mod -44.19977917334686
lat_025_mod -44.020277417553295
lat_025_mod -43.84023221817254
lat_025_mod -43.65964039861644
lat_025_mod -43.47850540339607
lat_025_mod -43.29682412001572
lat_025_mod -43.1146000178209
lat_0

lat_025_mod 10.31879437379502
lat_025_mod 10.564654234066332
lat_025_mod 10.810318694641733
lat_025_mod 11.055780978228533
lat_025_mod 11.301039171610363
lat_025_mod 11.54608653048691
lat_025_mod 11.790921183462055
lat_025_mod 12.035536420458884
lat_025_mod 12.279930413485692
lat_025_mod 12.524096487886421
lat_025_mod 12.768032860624551
lat_025_mod 13.011732893627801
lat_025_mod 13.255194850331272
lat_025_mod 13.498412130373907
lat_025_mod 13.741383045143644
lat_025_mod 13.984101033081737
lat_025_mod 14.22656445497414
lat_025_mod 14.468766789118314
lat_025_mod 14.710706447106602
lat_025_mod 14.952376948108617
lat_025_mod 15.193776755893923
lat_025_mod 15.434899431481558
lat_025_mod 15.675743492150964
lat_025_mod 15.916302541708545
lat_025_mod 16.156575152237487
lat_025_mod 16.396554971229563
lat_025_mod 16.636240626826247
lat_025_mod 16.875625811062203
lat_025_mod 17.114709209351908
lat_025_mod 17.35348455908949
lat_025_mod 17.591950604136887
lat_025_mod 17.830101128022857
lat_025_mod 

In [6]:
%%time
latS = -80
latN = 20
lonW = 90-360
lonE = 180-360
ekey='01deg'
expt = exptdata.exptdict[ekey]['expt']
print(expt)

tx_trans_z0_01_mod_month = cc.querying.getvar(expt=expt,
                                     session=session,
                                     ncfile='ocean_month.nc',
                                     variable='tx_trans_int_z').\
sel(yt_ocean=slice(latS,latN)).\
sel(xu_ocean=slice(lonW,lonE))
print(tx_trans_z0_01_mod_month)

tx_trans_z0_01_mod_month.load()
print(tx_trans_z0_01_mod_month)

xu_ocean = tx_trans_z0_01_mod_month.xu_ocean
xu_ocean_corrected = xu_ocean + 360
tx_trans_z0_01_mod_month = tx_trans_z0_01_mod_month.assign_coords(xu_ocean=xu_ocean_corrected)
print(tx_trans_z0_01_mod_month)

tx_trans_z0_01_mod = tx_trans_z0_01_mod_month.groupby('time.year').mean('time')
print(tx_trans_z0_01_mod)

tx_trans_z0_01_mod = tx_trans_z0_01_mod.sel(year=slice(1993,2017))
print(tx_trans_z0_01_mod)

tx_trans_z0_01_mod = tx_trans_z0_01_mod.cumsum('yt_ocean')
print(tx_trans_z0_01_mod)

<xarray.DataArray 'tx_trans_int_z' (year: 33, yt_ocean: 1422, xu_ocean: 900)>
array([[[            nan,             nan,             nan, ...,
                     nan,             nan,             nan],
        [            nan,             nan,             nan, ...,
                     nan,             nan,             nan],
        [            nan,             nan,             nan, ...,
                     nan,             nan,             nan],
        ...,
        [-1.04264616e+08, -7.44245280e+07, -5.68730280e+07, ...,
          4.45271200e+08,  3.48109792e+08,  2.77692576e+08],
        [-1.16053864e+08, -8.58174080e+07, -6.71617840e+07, ...,
          4.43354240e+08,  3.77638688e+08,  3.30823712e+08],
        [-1.20990984e+08, -1.01627416e+08, -8.23192720e+07, ...,
          4.46618240e+08,  3.99811680e+08,  3.61577088e+08]],

       [[            nan,             nan,             nan, ...,
                     nan,             nan,             nan],
        [            nan,

NameError: name 'tx_trans_z0_1_mod' is not defined

In [8]:
year = tx_trans_z0_01_mod.year
lat_01_mod = tx_trans_z0_01_mod.yt_ocean
lon_01_mod = tx_trans_z0_01_mod.xu_ocean

tx_trans_z0_01_trans_mod = tx_trans_z0_01_mod
print(np.shape(tx_trans_z0_01_trans_mod))
tx_trans_z0_01_slope_mod = xr.DataArray(
    np.zeros([np.shape(lat_01_mod)[0], np.shape(lon_01_mod)[0]]), dims=(
    'lat_01', 'lon_01'), coords=[lat_01_mod, lon_01_mod], name='tx_trans_z0_01')
tx_trans_z0_01_p_value_mod = xr.DataArray(
    np.zeros([np.shape(lat_01_mod)[0], np.shape(lon_01_mod)[0]]), dims=(
    'lat_01', 'lon_01'), coords=[lat_01_mod, lon_01_mod], name='tx_trans_z0_01')
tx_trans_z0_01_std_err_mod = xr.DataArray(
    np.zeros([np.shape(lat_01_mod)[0], np.shape(lon_01_mod)[0]]), dims=(
    'lat_01', 'lon_01'), coords=[lat_01_mod, lon_01_mod], name='tx_trans_z0_01')
for iid, i in enumerate(lat_01_mod):
    for jid, j in enumerate(lon_01_mod):
        tx_trans_z0_01_slope_mod[iid, jid], _, _, tx_trans_z0_01_p_value_mod[iid, jid], tx_trans_z0_01_std_err_mod[iid, jid] = \
        stats.linregress(year, tx_trans_z0_01_trans_mod[:, iid, jid])
    print('lat_01_mod ' + str(np.array(i)))
print(tx_trans_z0_01_slope_mod)

(25, 1422, 900)
lat_01_mod -79.96816911008175
lat_01_mod -79.92592975570126
lat_01_mod -79.88369040132078
lat_01_mod -79.84145104694029
lat_01_mod -79.79921169255981
lat_01_mod -79.75697233817932
lat_01_mod -79.71473298379884
lat_01_mod -79.67249362941835
lat_01_mod -79.63025427503787
lat_01_mod -79.58801492065739
lat_01_mod -79.5457755662769
lat_01_mod -79.50353621189642
lat_01_mod -79.46129685751593
lat_01_mod -79.41905750313545
lat_01_mod -79.37681814875496
lat_01_mod -79.33457879437448
lat_01_mod -79.292339439994
lat_01_mod -79.25010008561351
lat_01_mod -79.20786073123303
lat_01_mod -79.16562137685254
lat_01_mod -79.12338202247206
lat_01_mod -79.08114266809157
lat_01_mod -79.03890331371109
lat_01_mod -78.9966639593306
lat_01_mod -78.95442460495012
lat_01_mod -78.91218525056964
lat_01_mod -78.86994589618915
lat_01_mod -78.82770654180867
lat_01_mod -78.78546718742818
lat_01_mod -78.7432278330477
lat_01_mod -78.70098847866721
lat_01_mod -78.65874912428673
lat_01_mod -78.61650976990624

lat_01_mod -68.39458600982901
lat_01_mod -68.35234665544853
lat_01_mod -68.31010730106804
lat_01_mod -68.26786794668756
lat_01_mod -68.22562859230707
lat_01_mod -68.18338923792659
lat_01_mod -68.1411498835461
lat_01_mod -68.09891052916562
lat_01_mod -68.05667117478514
lat_01_mod -68.01443182040465
lat_01_mod -67.97219246602417
lat_01_mod -67.92995311164368
lat_01_mod -67.8877137572632
lat_01_mod -67.84547440288272
lat_01_mod -67.80323504850223
lat_01_mod -67.76099569412175
lat_01_mod -67.71875633974126
lat_01_mod -67.67651698536078
lat_01_mod -67.63427763098029
lat_01_mod -67.59203827659981
lat_01_mod -67.54979892221932
lat_01_mod -67.50755956783884
lat_01_mod -67.46532021345836
lat_01_mod -67.42308085907787
lat_01_mod -67.38084150469739
lat_01_mod -67.3386021503169
lat_01_mod -67.29636279593642
lat_01_mod -67.25412344155593
lat_01_mod -67.21188408717545
lat_01_mod -67.16964473279496
lat_01_mod -67.12740537841448
lat_01_mod -67.085166024034
lat_01_mod -67.04292666965351
lat_01_mod -67.

lat_01_mod -55.52266844284527
lat_01_mod -55.46601977374587
lat_01_mod -55.40928946300711
lat_01_mod -55.352477717530064
lat_01_mod -55.295584207874
lat_01_mod -55.23860914063544
lat_01_mod -55.18155218761087
lat_01_mod -55.124413555101775
lat_01_mod -55.067192916144876
lat_01_mod -55.00989047675615
lat_01_mod -54.95250591121559
lat_01_mod -54.89503942526325
lat_01_mod -54.83749069442549
lat_01_mod -54.77985992417602
lat_01_mod -54.722146791290726
lat_01_mod -54.66435150098657
lat_01_mod -54.60647373129214
lat_01_mod -54.54851368717732
lat_01_mod -54.49047104792663
lat_01_mod -54.43234601827253
lat_01_mod -54.37413827875874
lat_01_mod -54.315848033889985
lat_01_mod -54.25747496547251
lat_01_mod -54.19901927779302
lat_01_mod -54.14048065392365
lat_01_mod -54.08185929794283
lat_01_mod -54.02315489419197
lat_01_mod -53.96436764655098
lat_01_mod -53.90549724063403
lat_01_mod -53.84654388013229
lat_01_mod -53.78750725193614
lat_01_mod -53.728387559557866
lat_01_mod -53.66918449116759
lat_01

lat_01_mod -36.978305953662606
lat_01_mod -36.89837773550228
lat_01_mod -36.81836562198684
lat_01_mod -36.73826987181258
lat_01_mod -36.65809036388485
lat_01_mod -36.57782735795537
lat_01_mod -36.497480734781504
lat_01_mod -36.41705075517847
lat_01_mod -36.33653730175905
lat_01_mod -36.255940636409974
lat_01_mod -36.17526064360279
lat_01_mod -36.09449758630366
lat_01_mod -36.013651350846175
lat_01_mod -35.93272220128377
lat_01_mod -35.85171002581527
lat_01_mod -35.77061508958916
lat_01_mod -35.689437282672614
lat_01_mod -35.60817687131681
lat_01_mod -35.52683374746036
lat_01_mod -35.4454081784647
lat_01_mod -35.36390005814284
lat_01_mod -35.28230965497398
lat_01_mod -35.200636864648395
lat_01_mod -35.11888195677047
lat_01_mod -35.037044828910574
lat_01_mod -34.95512575180556
lat_01_mod -34.8731246249086
lat_01_mod -34.79104172009625
lat_01_mod -34.70887693870714
lat_01_mod -34.626630553764656
lat_01_mod -34.54430246849541
lat_01_mod -34.46189295707665
lat_01_mod -34.379401924625455
lat

lat_01_mod -12.841135115088473
lat_01_mod -12.743617161846737
lat_01_mod -12.646061734990322
lat_01_mod -12.548468946861613
lat_01_mod -12.450839222530183
lat_01_mod -12.35317267552425
lat_01_mod -12.25546973185579
lat_01_mod -12.157730506223396
lat_01_mod -12.059955425569713
lat_01_mod -11.962144605748115
lat_01_mod -11.864298474620044
lat_01_mod -11.766417149177911
lat_01_mod -11.668501058189939
lat_01_mod -11.570550319771696
lat_01_mod -11.47256536358603
lat_01_mod -11.374546308855647
lat_01_mod -11.27649358612572
lat_01_mod -11.178407315709944
lat_01_mod -11.080287929023381
lat_01_mod -10.982135547454417
lat_01_mod -10.883950603275439
lat_01_mod -10.785733218933103
lat_01_mod -10.687483827544405
lat_01_mod -10.589202552597722
lat_01_mod -10.490889828041823
lat_01_mod -10.392545778390115
lat_01_mod -10.294170838410169
lat_01_mod -10.195765133623615
lat_01_mod -10.09732909960372
lat_01_mod -9.998862862863398
lat_01_mod -9.900366859768386
lat_01_mod -9.801841217805821
lat_01_mod -9.70

lat_01_mod 14.492936722814687
lat_01_mod 14.589733467631538
lat_01_mod 14.686487587203125
lat_01_mod 14.783198981863242
lat_01_mod 14.879867237639997
lat_01_mod 14.976492256213264
lat_01_mod 15.073073624682506
lat_01_mod 15.16961124608732
lat_01_mod 15.26610470860825
lat_01_mod 15.36255391665811
lat_01_mod 15.458958459508088
lat_01_mod 15.555318242957538
lat_01_mod 15.651632857377699
lat_01_mod 15.747902209967636
lat_01_mod 15.84412589220787
lat_01_mod 15.940303812710189
lat_01_mod 16.036435564073468
lat_01_mod 16.13252105633506
lat_01_mod 16.228559883221106
lat_01_mod 16.32455195620721
lat_01_mod 16.42049687015551
lat_01_mod 16.51639453799239
lat_01_mod 16.61224455572456
lat_01_mod 16.708046837741524
lat_01_mod 16.803800981202976
lat_01_mod 16.899506901973744
lat_01_mod 16.99516419837473
lat_01_mod 17.090772787758112
lat_01_mod 17.18633226961407
lat_01_mod 17.281842562793994
lat_01_mod 17.37730326796523
lat_01_mod 17.47271430549008
lat_01_mod 17.56807527722078
lat_01_mod 17.6633861050

In [9]:
# tx_trans_z0_1_mod_xr = xr.DataArray(tx_trans_z0_1_mod.transpose('yt_ocean', 'xu_ocean', 'year'), name='tx_trans_z0_1_mod',
#                         coords=[lat_1_mod, lon_1_mod, year], 
#                         dims=['yt_ocean', 'xu_ocean', 'year'])
# print(tx_trans_z0_1_mod_xr)
# tx_trans_z0_1_mod_xr.to_netcdf(output_path + 'tx_trans_z0_1_mod.nc')

# tx_trans_z0_1_slope_mod_xr = xr.DataArray(tx_trans_z0_1_slope_mod, name='tx_trans_z0_1_slope_mod',
#                         coords=[lat_1_mod, lon_1_mod], 
#                         dims=['yt_ocean', 'xu_ocean'])
# print(tx_trans_z0_1_slope_mod_xr)
# tx_trans_z0_1_slope_mod_xr.to_netcdf(output_path + 'tx_trans_z0_1_slope_mod.nc')

# tx_trans_z0_1_p_value_mod_xr = xr.DataArray(tx_trans_z0_1_p_value_mod, name='tx_trans_z0_1_p_value_mod',
#                         coords=[lat_1_mod, lon_1_mod], 
#                         dims=['yt_ocean', 'xu_ocean'])
# print(tx_trans_z0_1_p_value_mod_xr)
# tx_trans_z0_1_p_value_mod_xr.to_netcdf(output_path + 'tx_trans_z0_1_p_value_mod.nc')

# tx_trans_z0_1_std_err_mod_xr = xr.DataArray(tx_trans_z0_1_std_err_mod, name='tx_trans_z0_1_std_err_mod',
#                         coords=[lat_1_mod, lon_1_mod], 
#                         dims=['yt_ocean', 'xu_ocean'])
# print(tx_trans_z0_1_std_err_mod_xr)
# tx_trans_z0_1_std_err_mod_xr.to_netcdf(output_path + 'tx_trans_z0_1_std_err_mod.nc')



# tx_trans_z0_025_mod_xr = xr.DataArray(tx_trans_z0_025_mod.transpose('yt_ocean', 'xu_ocean', 'year'), name='tx_trans_z0_025_mod',
#                         coords=[lat_025_mod, lon_025_mod, year], 
#                         dims=['yt_ocean', 'xu_ocean', 'year'])
# print(tx_trans_z0_025_mod_xr)
# tx_trans_z0_025_mod_xr.to_netcdf(output_path + 'tx_trans_z0_025_mod.nc')

# tx_trans_z0_025_slope_mod_xr = xr.DataArray(tx_trans_z0_025_slope_mod, name='tx_trans_z0_025_slope_mod',
#                         coords=[lat_025_mod, lon_025_mod], 
#                         dims=['yt_ocean', 'xu_ocean'])
# print(tx_trans_z0_025_slope_mod_xr)
# tx_trans_z0_025_slope_mod_xr.to_netcdf(output_path + 'tx_trans_z0_025_slope_mod.nc')

# tx_trans_z0_025_p_value_mod_xr = xr.DataArray(tx_trans_z0_025_p_value_mod, name='tx_trans_z0_025_p_value_mod',
#                         coords=[lat_025_mod, lon_025_mod], 
#                         dims=['yt_ocean', 'xu_ocean'])
# print(tx_trans_z0_025_p_value_mod_xr)
# tx_trans_z0_025_p_value_mod_xr.to_netcdf(output_path + 'tx_trans_z0_025_p_value_mod.nc')

# tx_trans_z0_025_std_err_mod_xr = xr.DataArray(tx_trans_z0_025_std_err_mod, name='tx_trans_z0_025_std_err_mod',
#                         coords=[lat_025_mod, lon_025_mod], 
#                         dims=['yt_ocean', 'xu_ocean'])
# print(tx_trans_z0_025_std_err_mod_xr)
# tx_trans_z0_025_std_err_mod_xr.to_netcdf(output_path + 'tx_trans_z0_025_std_err_mod.nc')



tx_trans_z0_01_mod_xr = xr.DataArray(tx_trans_z0_01_mod.transpose('yt_ocean', 'xu_ocean', 'year'), name='tx_trans_z0_01_mod',
                        coords=[lat_01_mod, lon_01_mod, year], 
                        dims=['yt_ocean', 'xu_ocean', 'year'])
print(tx_trans_z0_01_mod_xr)
tx_trans_z0_01_mod_xr.to_netcdf(output_path + 'tx_trans_z0_01_mod.nc')

tx_trans_z0_01_slope_mod_xr = xr.DataArray(tx_trans_z0_01_slope_mod, name='tx_trans_z0_01_slope_mod',
                        coords=[lat_01_mod, lon_01_mod], 
                        dims=['yt_ocean', 'xu_ocean'])
print(tx_trans_z0_01_slope_mod_xr)
tx_trans_z0_01_slope_mod_xr.to_netcdf(output_path + 'tx_trans_z0_01_slope_mod.nc')

tx_trans_z0_01_p_value_mod_xr = xr.DataArray(tx_trans_z0_01_p_value_mod, name='tx_trans_z0_01_p_value_mod',
                        coords=[lat_01_mod, lon_01_mod], 
                        dims=['yt_ocean', 'xu_ocean'])
print(tx_trans_z0_01_p_value_mod_xr)
tx_trans_z0_01_p_value_mod_xr.to_netcdf(output_path + 'tx_trans_z0_01_p_value_mod.nc')

tx_trans_z0_01_std_err_mod_xr = xr.DataArray(tx_trans_z0_01_std_err_mod, name='tx_trans_z0_01_std_err_mod',
                        coords=[lat_01_mod, lon_01_mod], 
                        dims=['yt_ocean', 'xu_ocean'])
print(tx_trans_z0_01_std_err_mod_xr)
tx_trans_z0_01_std_err_mod_xr.to_netcdf(output_path + 'tx_trans_z0_01_std_err_mod.nc')

<xarray.DataArray 'tx_trans_z0_01_mod' (yt_ocean: 1422, xu_ocean: 900, year: 25)>
array([[[0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        ...,
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00]],

       [[0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
 